# EXTRACTION DE KEYWORDS POUR L'AMOUR ATYPIQUE
### Corpus CAMille - Presse belge francophone (1831-1993)


## Imports

In [15]:
import os
import yake
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import spacy
from tqdm import tqdm  # Pour la barre de progression

In [16]:
# Chargement modèle spaCy pour le français (pour post-traitement)
nlp = spacy.load("fr_core_news_lg")

## Extraire les mots clés d'un document avec Yake

### Configuration de l'extracteur yake

In [17]:
# Paramètres optimisés pour un corpus historique/journalistique
kw_extractor = yake.KeywordExtractor(
    lan="fr",          # Langue française
    n=3,               # Taille max des n-grammes (1=unigram, 2=bigram, 3=trigram)
    dedupLim=0.9,      # Seuil de déduplication (0.9 = très strict)
    dedupFunc='seqm',  # Méthode de déduplication (séquence matching)
    windowsSize=2,     # Taille de la fenêtre contextuelle
    top=100,           # Top 100 keywords par document
    features=None      # Utilise les paramètres par défaut
)

In [20]:
data_path = "../../data/clean_corpus.txt"

with open(data_path, "r", encoding="utf-8") as f:
    full_text = f.read()


#### Fonction de nettoyage et filtrage

In [11]:
def filtrer_keywords(keywords):
    mots_exclus = {
        "gay", "bruxelles", "bruxellois", "bruxelloise", "charleroi", "liège", "anvers",
        "paris", "londres", "allemand", "français", "belge", "européen",
        "trans", "transport", "transports", "transporter",
        "rue", "place", "ville", "pays", "année", "temps", "jour", "heures",
        "hmu", "xii", "xix", "xxe", "xviii", "xvi", "xv", "xiv", "xiii",
        "—", "–", "…", "''", "``", "’", "‘", "“", "”", "«", "»"
    }

    mots_cibles = {
        "amour", "mariage", "relation", "liaison", "couple", "union", "époux", "épouse",
        "fiancé", "fiancée", "passion", "désir", "affection", "sentiment",
        "homosexuel", "homosexuelle", "transgenre", "transsexuel", "changement", "genre",
        "interdit", "secret", "scandale", "moral", "immoral", "crime", "délit",
        "condamnation", "libre", "atypique", "différent", "autre", "déviant",
        "anormal", "normal", "affectif", "famille", "société", "église",
        "catholique", "loi", "légal", "illégal", "tolérance", "acceptation",
        "rejet", "stigmate", "péchés", "vice", "honte", "caché", "clandestin",
        "concubinage", "cohabitation", "partenaire", "compagnon", "compagne",
        "divorce", "séparation", "adultère", "maîtresse", "amant", "amante"
    }

    keywords_filtres = []
    for kw, score in keywords:
        kw_lower = kw.lower()
        if any(exclu in kw_lower for exclu in mots_exclus):
            continue
        if any(cible in kw_lower for cible in mots_cibles):
            keywords_filtres.append((kw, score))
            continue
        doc = nlp(kw)
        if any(ent.label_ in ("PER", "ORG", "LOC", "GPE") for ent in doc.ents):
            continue
        keywords_filtres.append((kw, score * 1.5))
    return sorted(keywords_filtres, key=lambda x: x[1])

In [13]:
for f in sorted(files):
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    text = nettoyer_texte(text)
    keywords = kw_extractor.extract_keywords(text)
    keywords_filtres = filtrer_keywords(keywords)  # <-- Appel corrigé
    print(f"{f}: {keywords_filtres[:5]}")  # Affiche top 5 keywords filtrés

NameError: name 'files' is not defined

In [13]:
# Les dix premiers fichiers
files[:10]

['KB_JB838_1906-11-12_01-00004.txt',
 'KB_JB838_1970-06-13_01-00012.txt',
 'KB_JB838_1922-01-02_01-00007.txt',
 'KB_JB838_1952-02-29_01-00010.txt',
 'KB_JB838_1960-10-30_01-00007.txt',
 'KB_JB838_1966-08-30_01-00011.txt',
 'KB_JB838_1923-07-29_01-00003.txt',
 'KB_JB838_1893-11-28_01-00004.txt',
 'KB_JB838_1932-06-03_01-00011.txt',
 'KB_JB838_1961-12-30_01-00015.txt']

In [14]:
# Choisir un fichier
this_file = files[0]
this_file

'KB_JB838_1906-11-12_01-00004.txt'

In [15]:
# Récupérer le texte du fichier
text = open(os.path.join(data_path, this_file), 'r', encoding='utf-8').read()
text[:500]

"mi imnri r i «i i HMU ' î/tx-'l : Marché tenu hors villa, la 9. — U a été vaain Si téicj »M races indigènes de fr. 31<) à 5S'k 131 de. rasa îicHaKdui'te, do (r. 3S0 h 710. taureaux iallsènas,>ia ù\\ — à — ; 0II. hollandais, dufr. 0. — à 9.— la îdto- Vachei laitières: Bn vante 1Q. vendues 3\\ au prix la 410 à • «i h\\; génisses, Kl. '.9. i l. 2 i. id. da 370 i 6lütr. Marché a<u porcs. — Catégorie de lt ilashtya: ‘237 on vente; vendus 1 M.do ‘2 i.— à ;:, L —;i.l. des t'innlroV- I3ie;» vente, vendus 9"

In [16]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

[('maison', np.float64(0.003077658230936859)),
 ('rue', np.float64(0.0030922660251063282)),
 ('Maison de rentier', np.float64(0.007451367309712659)),
 ('Bruxelles', np.float64(0.00835766510324691)),
 ('Maison de commerce', np.float64(0.008414175763105169)),
 ('contenant', np.float64(0.010937362215490024)),
 ('centiares', np.float64(0.012015936317427473)),
 ('Demain lundi', np.float64(0.012279832515679162)),
 ('notaire', np.float64(0.0123644825432676)),
 ('Vendue', np.float64(0.016154270207986227)),
 ('louée', np.float64(0.01619773308081534)),
 ('lundi', np.float64(0.0163930958621132)),
 ('prix', np.float64(0.018093579312602026)),
 ('rentier', np.float64(0.019894369366872527)),
 ('are', np.float64(0.020215747658765645)),
 ('commerce', np.float64(0.02922268570960821)),
 ('ruo', np.float64(0.032678799542358944)),
 ('lieu', np.float64(0.03290812001961487)),
 ('rue Haute', np.float64(0.0371357795514965)),
 ('demain lundi Carmen', np.float64(0.037591252103167105)),
 ('rue Gallait', np.float6

In [17]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

['Demain lundi',
 'rue Haute',
 'rue Gallait',
 'maison mortuaire',
 'Notaire Van',
 'maisons qu’elle',
 'd’une maison',
 'rue Villa',
 'SUPERBE MAISON']

## Faire la même opération sur tous les documents

In [18]:
for f in sorted(files)[:10]:
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

KB_JB838_1887-12-22_01-00001.txt mentions these keywords: EXEMPLAIRES distribués, assassins politiques, distribués gratuitement, lomération bruxelloise, bruxelloise et.en, ræge quotidien, QUOTIDIEN GRATUIT, GRATUIT Température, facteur rural, Température Baromètre...
KB_JB838_1887-12-22_01-00002.txt mentions these keywords: Ligue nalio, rester fidèle, Van Ophem...
KB_JB838_1887-12-22_01-00003.txt mentions these keywords: prince Guillaume, Finet critique, Pierrot Macabre, HEURE Vienne...
KB_JB838_1887-12-22_01-00004.txt mentions these keywords: demande place, fiVS ÉPICERIES, ÉPICERIES t'Tu, lie Cologne, Fille val., place Madou, Bon Marché, rue Scailquin, rue Plantin, sci rue, bon certifie, rue lie, rue Bines, exposition Bruxelles, place sach, Bruxelles Centre, BRUXELLES Rideauœ...
KB_JB838_1887-12-22_01-00005.txt mentions these keywords: rue Plantin, OBJETS PERDUS, PERDUS gravure, LOUER DEMANDES, grande rue, rue Blaes, rue Hôtel, prix Bruxelles, rue Goflhrl, wôporler rue, rue St-Micliel